In [ ]:
import lsdb
print(lsdb.__version__)
import hats
print(hats.__version__)
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from dask.distributed import Client
from nested_pandas import NestedDtype

import nested_pandas as npd

In [ ]:
comcam_x_milliquas_computed.to_parquet("/sdf/home/n/ncaplar/AGN/comcam_x_milliquas_computed.parquet") # The output file path


In [ ]:
comcam_x_milliquas_computed = npd.read_parquet("/sdf/home/n/ncaplar/AGN/comcam_x_milliquas_computed.parquet") # The input file path

In [ ]:
comcam_x_milliquas_computed

In [ ]:
COLORS = {'u': '#0c71ff',
 'g': '#49be61',
 'r': '#c61c00',
 'i': '#ffc200',
 'z': '#f341a2',
 'y': '#5d0000'}

def create_mag_errors(flux, flux_err):
    # Make sure everything is valid and flux ± err stays positive
    upper = flux + flux_err
    lower = flux - flux_err
    mask = (
        flux.notna()
        & flux_err.notna()
        & (flux > 0)
        & (upper > 0)
        & (lower > 0)
    )

    # Initialize with NaNs
    mag = np.full_like(flux, np.nan, dtype=float)
    mag_err = np.full_like(flux, np.nan, dtype=float)

    # Apply the AB mag conversion only to valid values
    valid_flux = flux[mask]
    valid_err = flux_err[mask]

    mag[mask] = u.nJy.to(u.ABmag, valid_flux)
    upper_mag = u.nJy.to(u.ABmag, valid_flux + valid_err)
    lower_mag = u.nJy.to(u.ABmag, valid_flux - valid_err)
    mag_err[mask] = -(upper_mag - lower_mag) / 2

    return pd.Series(mag, index=flux.index), pd.Series(mag_err, index=flux.index)

def plot_mag_scale(ax, lc, flux_col, flux_err_col, x_name, x_label, show_legend=False):
    mag_vals = []
    for band, color in COLORS.items():
        band_lc = lc.query(f"band == '{band}'")
        if band_lc.empty:
            continue
        mag, mag_err = create_mag_errors(band_lc[flux_col], band_lc[flux_err_col])
        ax.errorbar(
            band_lc[x_name],
            mag,
            mag_err,
            fmt="o",
            label=band,
            color=color,
            alpha=1,
            markersize=5,
            capsize=3,
            elinewidth=1,
        )
        mag_vals.extend(mag.dropna().values)
    ax.set_xlabel(x_label)
    ax.set_ylabel("Magnitude (AB)")
    ax.invert_yaxis()
    if show_legend:
        ax.legend(loc="lower right", fontsize=12)
    return mag_vals

def scale_mag_y_axis(ax, all_mags):
    if all_mags[0]:
        ymin, ymax = np.nanmin(all_mags[0]), np.nanmax(all_mags[0])
        for i in range(2):
            ax[0, i].set_ylim(ymax + 0.1, ymin - 0.1)

"""            
def scale_mag_y_axis(ax, all_mags):
    if all_mags[0]:
        # Flatten, drop NaNs, compute median
        mags = np.array(all_mags[0])
        mags = mags[np.isfinite(mags)]
        if len(mags) > 0:
            median_mag = np.median(mags)
            for i in range(2):
                ax[0, i].set_ylim(median_mag + 0.5, median_mag - 0.5)
"""
def plot_mag_lightcurves(ax, row):
    datasets = [
        ("scienceFlux", "DIA Source", row.diaSource_COM),
        ("psfFlux", "DIA Forced", row.diaObjectForcedSource_COM),
    ]
    all_mags = [[], []]
    for i, (flux_col, label, lc) in enumerate(datasets):
        flux_err_col = f"{flux_col}Err"
        ax[0, i].set_title(f"{flux_col} ({label})")

        all_mags[0].extend(
            plot_mag_scale(ax[0, i], lc, flux_col, flux_err_col, "midpointMjdTai", "MJD", show_legend=(i == 1))
        )
    return all_mags

# Now plot
for i in range(len(comcam_AGN_many_r)):
    row = comcam_AGN_many_r.iloc[i]
    fig, ax = plt.subplots(1, 2, figsize=(16, 4))  # Just one row now
    fig.suptitle(f"RA={row.ra_COM:.5f}, Dec={row.dec_COM:.5f}", fontsize=16)
    all_mags = plot_mag_lightcurves(np.expand_dims(ax, 0), row)  # Make ax 2D for consistency
    scale_mag_y_axis(np.expand_dims(ax, 0), all_mags)
    plt.tight_layout()
    plt.show()

In [ ]:

def calcSF(tclip, fclip, nbins=10):
    """
    Compute the structure function (SF) using bins with equal number of time-lagged pairs.

    Parameters
    ----------
    tclip : ndarray
        1D array of time values (sorted).
    fclip : ndarray
        1D array of flux or magnitude values.
    nbins : int
        Number of bins, each with equal number of pairs.

    Returns
    -------
    tau_median : ndarray
        Median time lag in each bin.
    SF : ndarray
        Structure function values.
    N : ndarray
        Number of pairs in each bin (should be ~equal).
    """
    dt_list = []
    df2_list = []

    for i in range(len(tclip)):
        dt = tclip[i+1:] - tclip[i]
        df2 = (fclip[i+1:] - fclip[i])**2
        dt_list.append(dt)
        df2_list.append(df2)

    all_dt = np.concatenate(dt_list)
    all_df2 = np.concatenate(df2_list)

    # Sort by time difference
    sorted_indices = np.argsort(all_dt)
    all_dt = all_dt[sorted_indices]
    all_df2 = all_df2[sorted_indices]

    total_pairs = len(all_dt)
    pairs_per_bin = total_pairs // nbins

    tau_median = []
    SF = []
    N = []

    for i in range(nbins):
        start = i * pairs_per_bin
        end = (i + 1) * pairs_per_bin if i < nbins - 1 else total_pairs
        dt_bin = all_dt[start:end]
        df2_bin = all_df2[start:end]

        if len(df2_bin) > 0:
            tau_median.append(np.median(dt_bin))
            SF.append(np.sqrt(np.mean(df2_bin)))
            N.append(len(df2_bin))
        else:
            tau_median.append(np.nan)
            SF.append(np.nan)
            N.append(0)

    return np.array(tau_median), np.array(SF), np.array(N)

In [ ]:
len(row.diaObjectForcedSource_COM)

In [ ]:
len(comcam_AGN_many_r)

In [ ]:
i=5
row = comcam_AGN_many_r.iloc[i]
row_r = row.diaObjectForcedSource_COM[row.diaObjectForcedSource_COM['band'] == 'r']
row_r_clean = row_r[row_r['psfMag'].notna() & row_r['psfMagErr'].notna()]
# Identify all columns that contain 'flag' (case-insensitive)
flag_cols = [col for col in row_r_clean.columns if 'flag' in col.lower()]

# Keep only rows where all flag columns are False (or not True)
row_r_clean = row_r_clean[~row_r_clean[flag_cols].any(axis=1)]

tau, SF, N = calcSF(row_r_clean['midpointMjdTai'].values, row_r_clean['psfMag'].values, 10)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 5))

for i in range(15):
    row = comcam_AGN_many_r.iloc[i]
    row_r = row.diaObjectForcedSource_COM[row.diaObjectForcedSource_COM['band'] == 'r']
    row_r_clean = row_r[
        row_r['psfMag'].notna() & row_r['psfMagErr'].notna()
    ].copy()

    # Remove rows with any flags set
    flag_cols = [col for col in row_r_clean.columns if 'flag' in col.lower()]
    if flag_cols:
        row_r_clean = row_r_clean[~row_r_clean[flag_cols].any(axis=1)]

    # Skip if insufficient data
    if len(row_r_clean) < 2:
        continue

    # Compute SF
    tau, SF, N = calcSF(row_r_clean['midpointMjdTai'].values, row_r_clean['psfMag'].values, 10)

    axs[1].plot(tau, SF, '-o', label=f"Object {i}")

# Optional: plot reference models
tmodel = np.arange(0.1, 30, 0.1)
model1 = 0.1 * np.sqrt(tmodel / 100)
model2 = np.sqrt(0.05**2 + model1**2)
axs[1].plot(tmodel, model1, '--', color='gray', label="Model: A")
axs[1].plot(tmodel, model2, '--', color='black', label="Model: A + scatter")

# Labels and formatting
axs[1].set_ylim(bottom=0)
axs[1].set_xlabel("Time Lag (τ)")
axs[1].set_ylabel("Structure Function (SF)")
axs[1].set_title("Structure Functions (r-band)")
axs[1].grid(True)
axs[1].legend(fontsize=8)

# Optional: show one lightcurve on left for reference
row0 = comcam_AGN_many_r.iloc[0]
row0_r = row0.diaObjectForcedSource_COM[row0.diaObjectForcedSource_COM['band'] == 'r']
row0_r_clean = row0_r[row0_r['psfMag'].notna() & row0_r['psfMagErr'].notna()]
flag_cols = [col for col in row0_r_clean.columns if 'flag' in col.lower()]
if flag_cols:
    row0_r_clean = row0_r_clean[~row0_r_clean[flag_cols].any(axis=1)]
axs[0].scatter(row0_r_clean['midpointMjdTai'].values, row0_r_clean['psfMag'].values)
axs[0].set_xlabel("MJD")
axs[0].set_ylabel("psfMag")
axs[0].invert_yaxis()
axs[0].grid(True)
axs[0].set_title("Example Light Curve")

plt.tight_layout()
plt.show()